In [1]:
import pandas as pd
import numpy as np

unswap atoms

In [4]:
base = r"./comb/"
filename = base + "data.npt"
fopen = open(filename,"r")
flines = fopen.readlines()
fopen.close()

In [6]:
bonds_map = list()
for _ in flines[80535:120285]:
    tmp = []
    atomi = int(_.split()[2])
    atomj = int(_.split()[3])
    tmp=[min(atomi,atomj),max(atomi,atomj)]
    bonds_map.append(tmp)
    
bonds_map.sort()

In [8]:
def add_edge(graph, u, v):
    if u not in graph:
        graph[u] = []
    if v not in graph:
        graph[v] = []
    graph[u].append(v)
    graph[v].append(u)

def dfs(node, graph, visited, component):
    # Stack for the nodes to visit
    stack = [node]
    while stack:
        current = stack.pop()
        if current not in visited:
            visited.add(current)
            component.append(current)
            for neighbor in graph[current]:
                if neighbor not in visited:
                    stack.append(neighbor)

def find_connected_components(connections):
    graph = {}
    # Building the graph
    for u, v in connections:
        add_edge(graph, u, v)
    
    visited = set()
    components = []

    # Finding all components
    for node in graph:
        if node not in visited:
            component = []
            dfs(node, graph, visited, component)
            components.append(component)
    
    return components

# Example usage
connections = bonds_map
#components = find_connected_components(connections)
#for i, component in enumerate(components):
    #print(f"Group {i + 1}: {component}")

reshuffled_chains = find_connected_components(connections)

In [10]:
len(reshuffled_chains[-1])

160

In [12]:
swapedatom_df = pd.read_csv(base + "data.npt",skiprows=529,nrows=40000 ,sep="\s+",header=None)

In [14]:
swapedatom_df.columns=['atomid','molid','atype','q','x','y','z','nx','ny','nz']

In [16]:
swapedatom_df

,atomid,molid,atype,q,x,y,z,nx,ny,nz
0,7344,144,46,0,-17.104407,-15.483123,-16.595824,0,1,0
1,12913,113,81,0,-17.422569,-16.465823,-17.150659,1,1,0
2,17335,55,109,0,-16.718993,-16.513005,-14.367457,1,1,0
3,17409,129,109,0,-16.199152,-15.941772,-16.429598,1,1,0
4,13483,43,85,0,-17.525664,-8.833125,-14.634577,0,1,0
...,...,...,...,...,...,...,...,...,...,...
39995,39972,132,250,0,13.196246,12.101633,8.937778,0,1,-1
39996,34138,58,214,0,13.340304,3.475502,8.947482,0,1,-1
39997,33753,153,211,0,13.271608,6.499426,8.939222,-1,1,-1
39998,20148,148,126,0,15.964688,12.959147,9.011437,-1,0,-1


In [18]:
for ichain, chain in enumerate(reshuffled_chains):
#    atoms.query('atomid==@chain')['type']=ichain+1
    tmp = swapedatom_df.query('atomid==@chain')
    index_arr = tmp.index
    swapedatom_df.loc[index_arr,'atype']=ichain+1
sort_atoms = swapedatom_df.sort_values(by=['atype','molid'])

In [41]:
bb_list = [_ for _ in range(1,81)]
sc_lists = []
for i in range(20):
    sc_temp = [3 + i*4] + [_ for _ in range(81+i*4, 85+i*4)]
    sc_lists.append(sc_temp)

#sc_1 = [3] + [_ for _ in range(81, 93)]
#sc_2 = [7] + [_ for _ in range(93, 105)]
#sc_3 = [11] + [_ for _ in range(105, 117)]
#sc_4 = [15] + [_ for _ in range(117, 129)]
#sc_5 = [19] +[_ for _ in range(129, 141)]
#sc_6 = [23] +[_ for _ in range(141, 153)]
#sc_7 = [27] +[_ for _ in range(153, 165)]
#sc_7 = [31] +[_ for _ in range(165, 177)]
#sc_8 = [35] +[_ for _ in range(177, 189)]
#sc_9 = [39] +[_ for _ in range(189, 201)]
#sc_10 = [43] +[_ for _ in range(201, 213)]
#sc_11 = [43] +[_ for _ in range(201, 213)]
#sc_lists =[sc_1, sc_2, sc_3, sc_4, sc_5, sc_6, sc_7]

backbone_clean

In [47]:
for index_atype in range(1,251):
    pointer = 0;
    while pointer <= len(bb_list)-2:
        jdx = bb_list[pointer]
        jdx_1 = bb_list[pointer + 1]
        
        index_j = sort_atoms.query(f'atype=={index_atype} & molid=={jdx}').index
        index_k = sort_atoms.query(f'atype=={index_atype} & molid=={jdx_1}').index
        thisx = sort_atoms.loc[index_j,'x'].values[0]
        thatx = sort_atoms.loc[index_k,'x'].values[0]
        
        if thatx - thisx > 5:
            sort_atoms.loc[index_k,'x'] -= 17.804289634510873*2
        elif thatx - thisx < -5:
            sort_atoms.loc[index_k,'x'] += 17.804289634510873*2
        else:
            pass
        
        thisy = sort_atoms.loc[index_j,'y'].values[0]
        thaty = sort_atoms.loc[index_k,'y'].values[0]
        
        if thaty - thisy > 5:
            sort_atoms.loc[index_k,'y'] -= 17.804289634510873*2
        elif thaty - thisy < -5:
            sort_atoms.loc[index_k,'y'] += 17.804289634510873*2
        else:
            pass
        
        thisz = sort_atoms.loc[index_j,'z'].values[0]
        thatz = sort_atoms.loc[index_k,'z'].values[0]
        
        if thatz - thisz > 5:
            sort_atoms.loc[index_k,'z'] -= 17.804289634510873*2
        elif thatz - thisz < -5:
            sort_atoms.loc[index_k,'z'] += 17.804289634510873*2
        else:
            pass
        
        pointer += 1
        
    

In [51]:
###sidechain###
for index_atype in range(1,251):
    for index_of_sc in range(len(sc_lists)):
        sc_arr = sc_lists[index_of_sc]
        pointer = 0;
        while pointer < 4:
            jdx = sc_arr[pointer]
            jdx_1 = sc_arr[pointer + 1]
        
            index_j = sort_atoms.query(f'atype=={index_atype} & molid=={jdx}').index
            index_k = sort_atoms.query(f'atype=={index_atype} & molid=={jdx_1}').index
            thisx = sort_atoms.loc[index_j,'x'].values[0]
            thatx = sort_atoms.loc[index_k,'x'].values[0]
        
            if thatx - thisx > 5:
                sort_atoms.loc[index_k,'x'] -= 17.804289634510873*2
            elif thatx - thisx < -5:
                sort_atoms.loc[index_k,'x'] += 17.804289634510873*2
            else:
                pass
        
            thisy = sort_atoms.loc[index_j,'y'].values[0]
            thaty = sort_atoms.loc[index_k,'y'].values[0]
        
            if thaty - thisy > 5:
                sort_atoms.loc[index_k,'y'] -= 17.804289634510873*2
            elif thaty - thisy < -5:
                sort_atoms.loc[index_k,'y'] += 17.804289634510873*2
            else:
                pass
        
            thisz = sort_atoms.loc[index_j,'z'].values[0]
            thatz = sort_atoms.loc[index_k,'z'].values[0]
        
            if thatz - thisz > 5:
                sort_atoms.loc[index_k,'z'] -= 17.804289634510873*2
            elif thatz - thisz < -5:
                sort_atoms.loc[index_k,'z'] += 17.804289634510873*2
            else:
                pass
        
            pointer += 1


In [53]:
sort_atoms['molid'] = sort_atoms['atype']

In [55]:
sort_atoms

,atomid,molid,atype,q,x,y,z,nx,ny,nz
1955,1,1,1,0,-15.277157,-0.480414,5.710104,1,1,-1
1959,2,1,1,0,-15.539826,-0.910968,6.526660,1,1,-1
1940,3,1,1,0,-15.576905,-1.862841,6.821902,1,1,-1
1938,4,1,1,0,-16.087590,-2.568026,6.429268,1,1,-1
1951,5,1,1,0,-16.301786,-3.133061,5.635655,1,1,-1
...,...,...,...,...,...,...,...,...,...,...
18492,23516,250,250,0,0.067089,-2.900193,-1.826526,0,0,-1
18709,23517,250,250,0,0.767551,-2.011542,0.476336,0,0,-1
18255,23518,250,250,0,1.036505,-2.606974,-0.186183,0,0,-1
18275,23519,250,250,0,1.827007,-3.124489,-0.514809,0,0,-1


In [57]:
newid_list = []
for molid in range(1,251):
    temp_df = sort_atoms.query(f'atype == {molid}')
    atmid_arr = temp_df["atomid"].tolist()
    for _ in atmid_arr:
        if _%160 ==0:
            newid_list.append(molid*160)
        else:
            newid_list.append(_%160 + molid*160 - 160)

In [59]:
sort_atoms['atomid'] = newid_list

In [ ]:
newatype = [1]*len(newid_list)
sort_atoms['atype'] = newatype

In [61]:
sort_atoms

,atomid,molid,atype,q,x,y,z,nx,ny,nz
1955,1,1,1,0,-15.277157,-0.480414,5.710104,1,1,-1
1959,2,1,1,0,-15.539826,-0.910968,6.526660,1,1,-1
1940,3,1,1,0,-15.576905,-1.862841,6.821902,1,1,-1
1938,4,1,1,0,-16.087590,-2.568026,6.429268,1,1,-1
1951,5,1,1,0,-16.301786,-3.133061,5.635655,1,1,-1
...,...,...,...,...,...,...,...,...,...,...
18492,39996,250,250,0,0.067089,-2.900193,-1.826526,0,0,-1
18709,39997,250,250,0,0.767551,-2.011542,0.476336,0,0,-1
18255,39998,250,250,0,1.036505,-2.606974,-0.186183,0,0,-1
18275,39999,250,250,0,1.827007,-3.124489,-0.514809,0,0,-1


In [63]:
sort_atoms[['atomid','molid','atype','q','x','y','z']].to_csv(base + "/atoms",index=None,header=None,sep=" ")

In [ ]:
pd.set_option('display.max_rows', None)
tmp = sort_atoms.query('atype==26').copy()

In [ ]:
for temp_i in range(383):
    xxx = (tmp.x.values[temp_i] - tmp.x.values[temp_i+1])**2 + (tmp.y.values[temp_i] - tmp.y.values[temp_i+1])**2 + (tmp.z.values[temp_i] - tmp.z.values[temp_i+1])**2
    if xxx > 4.0:
        print(temp_i, xxx)

two filaments

In [ ]:
#base2 = r"/Volumes/DATA/3D_Print/DC/filaments_2024-04-14"
base2 = r"./"
atom_coord = pd.read_csv(base2 + "/3filament.dat",
                         header=None,skiprows=1029,nrows=192000,sep="\s+")

In [ ]:
atom_coord.columns = ['atid','molid','atype','q','x','y','z','nx','ny','nz']

In [ ]:
sort_atom_coord = atom_coord.sort_values(by = 'atid')

In [ ]:
sort_atom_coord

In [ ]:
newtype = [_//96000+1 for _ in range(len(sort_atom_coord))]

In [ ]:
sort_atom_coord['atype'] = newtype

In [ ]:
sort_atom_coord.to_csv(base2 + "/sort_atom.dat",header=None,index=None,sep=" ")

DC accelerate

In [ ]:
base3 = r"/Volumes/DATA/3D_Print/DC/print_2024-05-28/"
coords = pd.read_csv(base3 + "data.glass",header=None,skiprows=33,nrows=192000,sep="\s+")
coords.columns = ['atid','molid','atype','q','x','y','z','nx','ny','nz']

In [ ]:
sort_coords = coords.sort_values(by = 'atid')
sort_coords['atype'] = sort_coords['molid'].values

In [ ]:
newid_list = []
for _ in sort_coords['atid'].tolist():
    if _%320 ==0:
        newid_list.append(320)
    else:
        newid_list.append(_%320)

In [ ]:
sort_coords['molid'] = newid_list
sort_coords['atype'] = (sort_coords['atid'].values-1)//320 + 1

In [ ]:
sort_coords

In [ ]:
sort_coords.to_csv(base3 + "accelerate_merge.dat",header=None,index=None,sep=" ")